In [1]:
import os
import config
import pandas as pd
import time
print(config.root_path)

Config file loaded successfully
/Users/pradip.gupta/JEAP/object_detection


In [2]:
import findspark
findspark.init(config.spark_home)
from pyspark import SparkContext, SparkConf

try:    
    #start_spark()
    conf = SparkConf().setAppName('ObjectDetection').setMaster(config.spark_master)
    sc = SparkContext(conf=conf)
    sc.setLogLevel("ERROR")
except:
    sc.stop()
    conf = SparkConf().setAppName('ObjectDetection').setMaster(config.spark_master)
    sc = SparkContext(conf=conf)
    sc.setLogLevel("ERROR")

In [7]:
config.video_list_path
df = pd.read_excel(config.video_list_path)
video_list = df["links"]
video_url = video_list[2]
video_url = "https://www.youtube.com/watch?v=XQvcZWCacKE&pbjreload=10"
print(video_url)

http://www.youtube.com/watch?v=XQvcZWCacKE&pbjreload=10


In [10]:
from file_download import file_download
video_id, proto_name = file_download(video_url)
print(video_id,proto_name)

Video file  for url http://www.youtube.com/watch?v=XQvcZWCacKE&pbjreload=10 already present in database.                     Not downloading it
/Users/pradip.gupta/JEAP_database/video_library/3IdiotsRajusJobInterview.mp4 3IdiotsRajusJobInterview.bytes


In [7]:
from check_validity import check_validity
video_file_path = video_id
num_frames = check_validity(video_file_path)
num_frames

5059

In [8]:
import imageio
# Load the video frame by frame
videogen = imageio.get_reader(video_file_path,'ffmpeg')
fps = videogen.get_meta_data()['fps'] #fps to be used to decide min scene length

# Create a folder to store frames of the modified video
mod_folder, _ = proto_name.split(".")
modified_frames_path = os.path.join(config.modified_frames, mod_folder)
if not os.path.exists(modified_frames_path):
        os.mkdir(modified_frames_path) 

In [9]:
start = time.time()
import importlib
import extract_shots_with_pyscenedetect
importlib.reload(extract_shots_with_pyscenedetect)

from extract_shots_with_pyscenedetect import extract_shots_with_pyscenedetect
# Scene detection algorithm
scene_list, scene_list_tc = extract_shots_with_pyscenedetect(video_file_path, threshold = config.scene_detect_thresh, 
                                                             min_scene_len = 5*fps)
print("time taken is {}".format(time.time()-start))

[PySceneDetect] Parsing video 3IdiotsRajusJobInterview.mp4...
[PySceneDetect] Video Resolution / Framerate: 1280 x 720 / 29.970 FPS
Verify that the above parameters are correct (especially framerate, use --force-fps to correct if required).
No of scenes detected are 20
The scene changes are at the following time stamps ['00:00:00.000', '00:00:03.670', '00:00:12.479', '00:00:18.852', '00:00:27.894', '00:00:34.401', '00:00:44.310', '00:00:55.221', '00:01:11.571', '00:01:18.344', '00:01:29.389', '00:01:35.128', '00:01:43.937', '00:01:56.549', '00:02:06.092', '00:02:18.238', '00:02:25.178', '00:02:32.719', '00:02:39.759', '00:02:48.001', '00:02:48.735']
time taken is 91.65367197990417


In [10]:
sc.stop()